# 数据加载

Torch中的Dataset和Dataloader方法详解     
在PyTorch中，Dataset 和 DataLoader 是处理数据的核心工具，尤其适用于自定义数据集。Dataset 负责存储样本及其标签，而 DataLoader 则负责批量加载数据并提供迭代功能。

Dataset是一个抽象类，自定义数据集需要继承它，并实现以下两个方法：

- \_\_len__()：返回数据集的大小。

- \_\_getitem__()：根据索引返回一个样本（数据和标签）。

**自定义数据集示例**    
假设我们有一个自定义数据集，数据存储在 data 文件夹中，包含图像和对应的标签文件（如CSV或TXT）。

In [ ]:
import torch
from torch.utils.data import Dataset
import pandas as pd
from PIL import Image
import os

class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (str): 包含标签的CSV文件路径
            root_dir (str): 数据存储的根目录
            transform (callable, optional): 可选的预处理变换
        """
        self.labels = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels)  # 返回数据集的总样本数

    def __getitem__(self, idx):
        # 加载图像
        img_name = os.path.join(self.root_dir, self.labels.iloc[idx, 0])
        image = Image.open(img_name)  # 使用PIL加载图像

        # 加载标签
        label = self.labels.iloc[idx, 1]

        # 应用变换（如果有）
        if self.transform:
            image = self.transform(image)

        return image, label  # 返回样本和标签

DataLoader 负责批量加载数据，并提供以下功能：

- 批处理（batch_size）
- 数据打乱（shuffle=True）
- 多进程加载（num_workers）
- 自动内存管理（pin_memory 加速GPU训练）

In [ ]:
from torch.utils.data import DataLoader
from torchvision import transforms

# 定义数据变换
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 调整图像大小
    transforms.ToTensor(),          # 转为Tensor并归一化到 [0,1]
])

# 初始化自定义数据集
dataset = CustomDataset(
    csv_file="data/labels.csv",
    root_dir="data/images",
    transform=transform
)

# 创建DataLoader
dataloader = DataLoader(
    dataset,
    batch_size=32,      # 每批加载32个样本
    shuffle=True,        # 打乱数据顺序
    num_workers=4,       # 使用4个子进程加载数据
    pin_memory=True      # 加速GPU数据传输
)



使用DataLoader迭代调取数据训练有两种方法    
- For循环；更简洁常用
- iter()和next()结合；可手动控制迭代过程

In [ ]:
# 方法1
for batch_images, batch_labels in dataloader:
    print("Batch images shape:", batch_images.shape)  # [32, 3, 256, 256]
    print("Batch labels:", batch_labels)

# 方法2
data_iter = iter(dataloader)  # 显式创建迭代器

# 手动获取一个批次
batch_x, batch_y = next(data_iter)

# 或者循环部分批次
for _ in range(10):  # 只迭代10个批次
    batch_x, batch_y = next(data_iter)
    # 训练代码